In [1]:
import torch
import torch.nn as nn
from sinabs.from_torch import from_model
from sinabs.backend.dynapcnn import DynapcnnNetworkGraph
from sinabs.layers import Merge, IAFSqueeze

In [2]:
torch.manual_seed(0)

In [3]:
channels = 1
height = 28
width = 28

input_shape = (channels, height, width)

## Network Module (pure Pytorch)

In [4]:
class SNN(nn.Module):
    def __init__(self) -> None:
        super().__init__()

        self.conv1 = nn.Conv2d(1, 20, 5, 1, bias=False)
        self.iaf1 = IAFSqueeze(batch_size=1)
        self.pool1 = nn.AvgPool2d(2,2)

        self.conv2 = nn.Conv2d(20, 32, 5, 1, bias=False)
        self.iaf2 = IAFSqueeze(batch_size=1)
        self.pool2 = nn.AvgPool2d(2,2)

        self.conv3 = nn.Conv2d(32, 2, 3, 1, bias=False)
        self.iaf3 = IAFSqueeze(batch_size=1)
        self.pool3 = nn.AvgPool2d(2,2)

        self.flat = nn.Flatten()

        self.fc1 = nn.Linear(242, 500, bias=False)
        self.iaf4 = IAFSqueeze(batch_size=1)
        self.fc2 = nn.Linear(500, 10, bias=False)
        self.iaf5 = IAFSqueeze(batch_size=1)

        self.adder = Merge()

    def forward(self, x):
        
        con1_out = self.conv1(x)
        iaf1_out = self.iaf1(con1_out)
        pool1_out = self.pool1(iaf1_out)

        conv2_out = self.conv2(pool1_out)
        iaf2_out = self.iaf2(conv2_out)
        pool2_out = self.pool2(iaf2_out)

        conv3_out = self.conv3(self.adder(iaf1_out, pool2_out))
        iaf3_out = self.iaf3(conv3_out)
        pool3_out = self.pool3(iaf3_out)

        flat_out = self.flat(pool3_out)
        
        fc1_out = self.fc1(flat_out)
        iaf4_out = self.iaf4(fc1_out)
        fc2_out = self.fc2(iaf4_out)
        iaf5_out = self.iaf5(fc2_out)

        return iaf5_out

In [5]:
snn = SNN()

In [6]:
x = torch.randn((1, *input_shape))

con1_out = snn.conv1(x)
iaf1_out = snn.iaf1(con1_out)
print('iaf1_out: ', iaf1_out.shape)
pool1_out = snn.pool1(iaf1_out)

conv2_out = snn.conv2(pool1_out)
iaf2_out = snn.iaf2(conv2_out)
pool2_out = snn.pool2(iaf2_out)
print('pool2_out: ', pool2_out.shape)

added = snn.adder(iaf1_out, pool2_out)
print('added: ', added.shape)

conv3_out = snn.conv3(added)
iaf3_out = snn.iaf3(conv3_out)
pool3_out = snn.pool3(iaf3_out)
print('pool3_out: ', pool3_out.shape)

flat_out = snn.flat(pool3_out)
print('flat_out: ', flat_out.shape)

fc1_out = snn.fc1(flat_out)
iaf4_out = snn.iaf4(fc1_out)
fc2_out = snn.fc2(iaf4_out)
iaf5_out = snn.iaf5(fc2_out)

iaf1_out:  torch.Size([1, 20, 24, 24])
pool2_out:  torch.Size([1, 32, 4, 4])
added:  torch.Size([1, 32, 24, 24])
pool3_out:  torch.Size([1, 2, 11, 11])
flat_out:  torch.Size([1, 242])


## DynapCNN Model

In [7]:
hw_model = DynapcnnNetworkGraph(
    snn,
    discretize=True,
    input_shape=input_shape
)

In [8]:
print(hw_model)


layer index: 0
layer modules: DynapcnnLayer(
  (conv_layer): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1), bias=False)
  (spk_layer): IAFSqueeze(spike_threshold=Parameter containing:
  tensor(1.), min_v_mem=Parameter containing:
  tensor(-32768.), batch_size=1, num_timesteps=-1)
  (pool_layer): SumPool2d(norm_type=1, kernel_size=(2, 2), stride=None, ceil_mode=False)
)
layer destinations: [2, 1]
assigned core: -1

layer index: 1
layer modules: DynapcnnLayer(
  (conv_layer): Conv2d(20, 32, kernel_size=(5, 5), stride=(1, 1), bias=False)
  (spk_layer): IAFSqueeze(spike_threshold=Parameter containing:
  tensor(1.), min_v_mem=Parameter containing:
  tensor(-32768.), batch_size=1, num_timesteps=-1)
  (pool_layer): SumPool2d(norm_type=1, kernel_size=(2, 2), stride=None, ceil_mode=False)
)
layer destinations: [2]
assigned core: -1

layer index: 2
layer modules: DynapcnnLayer(
  (conv_layer): Conv2d(32, 2, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (spk_layer): IAFSqueeze(spike_thresho

In [ ]:
hw_model.to(device="speck2edevkit:0")